In [1]:
import requests
import json
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [2]:
def g(url):
    
    with requests.Session() as s:
        retries = Retry(total=10, backoff_factor=1, status_forcelist=[500, 502, 503, 504])

        s.mount('http://', HTTPAdapter(max_retries=retries))
        s.mount('https://', HTTPAdapter(max_retries=retries))

        return s.get(url, timeout=10)

In [3]:
sqlstr = lambda string: "'" + str(string).replace("'", "''") + "'"

### Results

In [4]:
## SQL create statements
stmts_faculty = [] # done
stmts_degree = []
stmts_subject = [] # done
stmts_course = [] # done
stmts_courseGroup = []
stmts_courseGroupMember = []
stmts_prerequisite = []
stmts_degreeRequirement = []
stmts_term = [] # done
stmts_courseOffering = [] # done

In [5]:
## Entity Sets for Later Use
set_faculty = set()
set_subject = set()
set_course = set()
set_courseOffering = set()

### Term

In [6]:
## strings for terms
terms = [str(100 + y) + str(t) for y in range (10, 21) for t in [1, 5, 9]]

In [7]:
stmts_term = [f'INSERT INTO term (code) VALUES ({term});' for term in terms]

### Faculty

In [8]:
with open('faculty.json', 'r') as faculty_f:
    faculties = json.load(faculty_f)

In [9]:
set_faculty = set()
stmts_faculty = []

for faculty in faculties:
    title = faculty['group_code']
    short_name = faculty['group_short_name']
    full_name = faculty['group_full_name']
    
    set_faculty.add(title)
    stmts_faculty.append(
        "INSERT INTO faculty (title, shortName, fullName) VALUES (%s, %s, %s);" % 
        (sqlstr(title), sqlstr(short_name), sqlstr(full_name))
    )

### Subject

In [10]:
with open('subject.json', 'r') as subject_f:
  subjects = json.load(subject_f)

In [11]:
set_subject = set()
stmts_subject = []

for subject in subjects:
    title = subject['subject']
    desc = subject['description']
    faculty = subject['group']
    
    if faculty in set_faculty:
        set_subject.add(title)
        stmts_subject.append(
            "INSERT INTO subject (title, facultyTitle, description) VALUES (%s, %s, %s);" % 
            (sqlstr(title), sqlstr(faculty), sqlstr(desc))
        )

### Courses

#### You can import the courses from `course.txt` if the file exists.

In [32]:
set_course = set()
with open('course.txt', 'r') as f:
    for line in f:
        set_course.add(line.strip())

#### Alternatively, you can scrape the course data from the OpenAPI as well.

In [12]:
courses_r = g('https://api.uwaterloo.ca/v2/courses.json?key=e5c5ea41b7715202b2846b04b286f29d')
courses = list(filter(lambda c: '`' not in c['catalog_number'], courses_r.json()['data'])) # there is a "GER 744`"

In [13]:
set_course = set()
stmts_course = []

l = len(courses)

for i, c in enumerate(courses):
    subject_title = c['subject']
    catalog_number = c['catalog_number']
    
    course_code = subject_title + ' ' + catalog_number
    
    ## print the progress of processing
    print('%s/%s course(s) processed. Processing %s.\t\t' % (i, l, course_code), end='\r', flush=True)

    course_detail_r = g(
        'https://api.uwaterloo.ca/v2/courses/%s/%s.json?key=e5c5ea41b7715202b2846b04b286f29d' %
        (subject_title, catalog_number)
    ).json()

    if 'data' in course_detail_r and course_detail_r['data'] and subject_title in set_subject:
        course_detail = course_detail_r['data']

        title = course_detail['title']
        credit = course_detail['units']
        desc = course_detail['description']
        course_types = ', '.join(ins for ins in course_detail['instructions'])

        set_course.add(course_code)
        stmts_course.append(
            "INSERT INTO course (courseCode, title, credit, courseTypes, description, subjectTitle) VALUES (%s, %s, %s, %s, %s, %s);" %
            (sqlstr(course_code),
             sqlstr(title),
             credit,
             sqlstr(course_types),
             sqlstr(desc),
             sqlstr(subject_title))
        )
        
print('\ndone.')

7448/7449 course(s) processed. Processing SYDE 263.				k.			
done.


### Course Offerings

In [35]:
set_courseOffering = set()
stmts_courseOffering = []

l = len(set_subject)
l_terms = len(terms)

for i, subject_title in enumerate(set_subject):
    
    for j, term in enumerate(terms):
        
        ## print the progress of processing
        print('%s/%s terms out of %s/%s subject(s) processed. Processing term=%s, subject=%s. So far %s offering(s) recorded.\t\t' %
              (j, l_terms, i, l, term, subject_title, len(stmts_courseOffering)), end='\r', flush=True)
            
        offerings_r = g(
            'https://api.uwaterloo.ca/v2/terms/%s/%s/schedule.json?key=e5c5ea41b7715202b2846b04b286f29d' %
            (term, subject_title)
        ).json()
                
        if 'data' in offerings_r and offerings_r['data']:
            offerings = offerings_r['data']

            for offering in offerings:
                catalog_number = offering['catalog_number']
                course_code = subject_title + ' ' + catalog_number

                if course_code in set_course:
                    args = []
                    vals = []

                    args.append('courseCode')
                    vals.append(sqlstr(course_code))

                    args.append('termCode')
                    vals.append(term)

                    section = offering['section'].split(' ')

                    courseType = section[0]
                    component = section[1]
                    
                    if (course_code, term, component) not in set_courseOffering:
                        args.append('courseType')
                        vals.append(sqlstr(courseType))

                        args.append('component')
                        vals.append(component)

                        if offering['related_component_1']:
                            args.append('relatedComponent1')
                            vals.append(str(offering['related_component_1']))

                        if offering['related_component_2']:
                            args.append('relatedComponent2')
                            vals.append(str(offering['related_component_2']))

                        enrlCap = offering['enrollment_capacity']
                        enrlTot = offering['enrollment_total']

                        args.append('enrlCap')
                        vals.append(str(enrlCap))

                        args.append('enrlTot')
                        vals.append(str(enrlTot))

                        classes = offering['classes']
                        if classes:
                            cls = classes[0]

                            instructors = cls['instructors']
                            if instructors:
                                prof = instructors[0].split(',')

                                if len(prof) > 1:
                                    args.append('profFirstName')
                                    vals.append(sqlstr(prof[1]))

                                args.append('profLastName')
                                vals.append(sqlstr(prof[0]))

                            clsDate = cls['date']
                            if clsDate:
                                if clsDate['start_time']:
                                    args.append('classStartTime')
                                    vals.append(sqlstr(clsDate['start_time']))

                                if clsDate['end_time']:
                                    args.append('classEndTime')
                                    vals.append(sqlstr(clsDate['end_time']))

                                if clsDate['weekdays']:
                                    args.append('classWeekdays')
                                    vals.append(sqlstr(clsDate['weekdays']))

                            loc = cls['location']
                            if loc:
                                if loc['building']:
                                    args.append('classBuilding')
                                    vals.append(sqlstr(loc['building']))

                                if loc['room']:
                                    args.append('classRoom')
                                    vals.append(sqlstr(loc['room']))

                        set_courseOffering.add((course_code, term, component))
                        stmts_courseOffering.append(
                            f"INSERT INTO courseOffering ({', '.join(args)}) VALUES ({', '.join(vals)});"
                        )
                    
print('\ndone.')

32/33 terms out of 206/207 subject(s) processed. Processing term=1209, subject=GENE. So far 90951 offering(s) recorded.				
done.


### Output to Files

In [20]:
with open('insert_1_faculty.sql', 'w+') as f:
    for line in stmts_faculty:
        f.write(line + '\n')

In [21]:
with open('insert_2_subject.sql', 'w+') as f:
    for line in stmts_subject:
        f.write(line + '\n')

In [22]:
with open('insert_3_course.sql', 'w+') as f:
    for line in stmts_course:
        f.write(line + '\n')

In [23]:
with open('insert_4_term.sql', 'w+') as f:
    for line in stmts_term:
        f.write(line + '\n')

In [36]:
with open('insert_5_courseOffering.sql', 'w+') as f:
    for line in stmts_courseOffering:
        f.write(line + '\n')

### Export Course.txt

In [24]:
with open('course.txt', 'w+') as f:
    for line in set_course:
        f.write(line + '\n')